<a href="https://colab.research.google.com/github/seenu-g/eva4-2/blob/master/week9/nlp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1 - Simple Sentiment Analysis¶

In [2]:
import torch
from torchtext import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField(dtype = torch.float)

In [3]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:05<00:00, 14.6MB/s]


In [4]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [5]:
print(vars(train_data.examples[0]))

{'text': ['This', 'film', 'opened', 'to', 'poor', 'showings', 'in', 'the', 'first', 'few', 'weeks', '.', 'Then', 'Meena', 'Kumari', 'died', 'and', 'it', 'just', 'brought', 'the', 'crowds', 'rolling', 'in', '.', 'Songs', 'on', 'All', 'India', 'Radio', ',', 'especially', 'Inhi', 'LogoN', 'ne', 'were', 'played', 'so', 'often', 'that', 'I', 'was', 'sick', 'of', 'them', 'at', 'the', 'time', ',', 'despite', 'recognising', 'their', 'beauty', '!', '<', 'br', '/><br', '/>Yes', ',', 'it', 'did', 'take', 'all', 'those', 'years', 'to', 'make', '.', 'This', 'was', 'because', 'the', 'marriage', 'was', 'a', 'very', 'unhappy', 'one', 'and', 'Kamal', 'Amrohi', 'also', 'had', 'difficulty', 'finding', 'the', 'money', 'to', 'make', 'the', 'film', ';', 'looking', 'at', 'the', 'sumptous', 'sets', 'and', 'costumes', ',', 'not', 'surprising', '!', '!', 'Not', 'only', 'does', 'Meena', 'Kumari', 'age', 'and', 'fall', 'ill', 'but', 'listen', 'carefully', 'to', 'Lata', "'s", 'voice', '.', 'Inhi', 'logoN', 'ne', '

In [6]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [7]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [8]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 203050), (',', 192823), ('.', 165678), ('and', 109957), ('a', 109264), ('of', 100930), ('to', 93500), ('is', 76092), ('in', 61480), ('I', 53837), ('it', 53663), ('that', 49375), ('"', 44101), ("'s", 43869), ('this', 42366), ('-', 37187), ('/><br', 35546), ('was', 35027), ('as', 30323), ('with', 29969)]


In [9]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [10]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f18f4876f28>, {'neg': 0, 'pos': 1})


In [11]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [13]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [14]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [15]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


In [16]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

In [17]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [22]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [23]:
  def evaluate(model, iterator, criterion):
  
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    count = 0
    with torch.no_grad():
    
        for batch in iterator:
            if (count <5):
              print(batch.text)
              count+=1
            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [24]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [25]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

tensor([[  66,   66,  612,  ...,   11,  238, 7734],
        [  24,   19,    2,  ...,   57,  306,    0],
        [ 318,    6, 2224,  ...,   30,  148,    9],
        ...,
        [  10,   16,    7,  ...,    1,    1,    1],
        [1823,   23,  323,  ...,    1,    1,    1],
        [   4,    4,   39,  ...,    1,    1,    1]])
tensor([[   25, 12028,    66,  ...,    56,   782,   127],
        [    0,     5,    23,  ...,    15, 11633,     6],
        [ 8734,  2390,    99,  ...,     6, 12421,     0],
        ...,
        [  893,  8364, 21473,  ...,     1,     1,     1],
        [  672,  3683,  1065,  ...,     1,     1,     1],
        [    4,     4,    58,  ...,     1,     1,     1]])
tensor([[  66,   66,  612,  ...,   11, 7953,   11],
        [   9,   23,    6,  ...,  176,    3,   19],
        [ 439,    9,  997,  ...,    6,   49,   62],
        ...,
        [ 956,    5, 2276,  ...,    1,    1,    1],
        [6107, 5935,    4,  ...,    1,    1,    1],
        [   0,    4,    1,  ...,    1, 

In [26]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

tensor([[  469,    11,    11,  ...,  6738,  7783,     0],
        [    3,    75,   161,  ...,     5,     2,     0],
        [   16,     8,   126,  ...,   771,   297,     0],
        ...,
        [10829,    74,  2311,  ...,     1,     1,     1],
        [    4,   732,   428,  ...,     1,     1,     1],
        [   14,     4,     4,  ...,     1,     1,     1]])
tensor([[   66,    25,   324,  ...,   149,    25,   261],
        [   23,   286,     2,  ...,    76,  6220,     6],
        [   19,    17, 14006,  ...,   834, 22417,  4736],
        ...,
        [ 5734,    80,   105,  ...,     1,     1,     1],
        [ 1419,  2745,    23,  ...,     1,     1,     1],
        [   39,     4,     4,  ...,     1,     1,     1]])
tensor([[ 1535,    25,    25,  ...,  4878,   469,   332],
        [  199,   672,  1260,  ...,   630,     3, 18184],
        [   14,   332,   344,  ...,     7,    11,   230],
        ...,
        [  472,     9,  2180,  ...,     1,     1,     1],
        [  635, 19167,     4,  